In [3]:
# Another AI generated code snippet & Explanation: Mthokozisi

HIV Transmission and Intervention Model
I've created a modified version of the original infection model that's specifically designed to simulate HIV transmission and test various intervention strategies. Here are the key changes and features:
HIV-Specific Characteristics Added

Disease Progression Stages:

Expanded the disease states from 3 (SIR) to 6 states:

Susceptible
HIV+ Untreated
HIV+ Treated
AIDS Untreated
AIDS Treated
Dead




Realistic Timeline:

Models the long-term nature of HIV (average ~10 years progression from infection to AIDS without treatment)
Uses daily time steps for more accurate modeling


Treatment Effects:

Includes testing rates (people discovering their status)
Treatment reduces transmission by ~96% (reflecting U=U principle: Undetectable = Untransmittable)
Treatment slows progression to AIDS and reduces mortality


Transmission Dynamics:

Variable transmission rates based on disease stage (higher in early infection and AIDS stage)
Condom use reduces transmission by ~95%
Sexual encounters modeled probabilistically


Data Collection:

Tracks prevalence, incidence (new infections), and mortality
Allows for comparison across different intervention scenarios



Testing Interventions
The model includes functions to test and compare different intervention strategies:

Baseline: 20% treatment coverage, 50% condom use
Increased Treatment: 90% treatment coverage, standard condom use
Increased Prevention: Standard treatment, 90% condom use
Combined Approach: 90% treatment coverage, 90% condom use

You can easily modify the parameters to test other scenarios like:

Pre-Exposure Prophylaxis (PrEP) by giving susceptible people protection
Behavioral interventions that reduce risky encounters
Education campaigns that increase testing rates

How to Use the Model
To run a specific intervention scenario:
python# Run a 20-year simulation with 2000 people
simulation_data = run_model_with_params(years=20, population=2000, treatment_coverage=0.5, condom_use=0.7)

# Plot the results
plot_hiv_results(simulation_data, "Custom HIV Intervention Scenario")
To compare multiple interventions:
pythonresults = compare_interventions()
Would you like me to explain any specific part of the model in more detail, or would you like to see additional features added to better simulate HIV transmission?

In [2]:
# Ethaton Attempt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import random

# Define the states of the agents
class State:
    SUSCEPTIBLE = 0
    INFECTED = 1
    RECOVERED = 2

# Define the agent class
class HIVAgent:
    """An agent in the HIV infection model."""
    
    def __init__(self, unique_id, model):
        self.unique_id = unique_id
        self.model = model
        self.state = State.SUSCEPTIBLE
        self.infection_time = 0
        self.pos = None  # Position in the network graph

    def step(self):
        """Agent's step in the model."""
        if self.state == State.INFECTED:
            self.infect_neighbors()
            self.check_recovery()

    def infect_neighbors(self):
        """Infect neighboring agents."""
        neighbors = list(self.model.graph.neighbors(self.pos))
        for neighbor_id in neighbors:
            neighbor = self.model.agents[neighbor_id]
            if neighbor.state == State.SUSCEPTIBLE:
                if random.random() < self.model.ptrans:
                    neighbor.state = State.INFECTED
                    neighbor.infection_time = self.model.time

    def check_recovery(self):
        """Check if the agent recovers."""
        if self.model.time - self.infection_time >= self.model.recovery_time:
            self.state = State.RECOVERED

# Define the model class
class HIVModel:
    """A model for HIV infection spread."""
    
    def __init__(self, N, ptrans, recovery_time):
        self.num_agents = N
        self.ptrans = ptrans
        self.recovery_time = recovery_time
        self.time = 0  # Manual time step tracking

        self.graph = nx.erdos_renyi_graph(N, 0.1)
        self.agents = []

        # Create and assign agents to nodes
        for node in self.graph.nodes():
            agent = HIVAgent(node, self)
            agent.pos = node
            self.agents.append(agent)

        # Infect a random agent at the start
        initial_infected = random.choice(self.agents)
        initial_infected.state = State.INFECTED
        initial_infected.infection_time = 0

        # Data collection
        self.data = []

    def step(self):
        """Advance the model by one step."""
        random.shuffle(self.agents)  # Randomize activation
        for agent in self.agents:
            agent.step()

        # Data collection
        for agent in self.agents:
            self.data.append({
                "Step": self.time,
                "AgentID": agent.unique_id,
                "State": agent.state
            })
        self.time += 1

# Parameters
N = 100  # Number of agents
ptrans = 0.1  # Probability of transmission
recovery_time = 10  # Time until recovery

# Create and run the model
model = HIVModel(N, ptrans, recovery_time)
steps = 50
for i in range(steps):
    model.step()

# Convert collected data to DataFrame
data = pd.DataFrame(model.data)

# Plot the results
plt.figure(figsize=(10, 6))
for state, label in zip([State.SUSCEPTIBLE, State.INFECTED, State.RECOVERED],
                        ['Susceptible', 'Infected', 'Recovered']):
    count_by_step = data[data['State'] == state].groupby('Step').size()
    plt.plot(count_by_step, label=label)

plt.xlabel('Time Steps')
plt.ylabel('Number of Agents')
plt.title('HIV Infection Spread Simulation')
plt.legend()
plt.tight_layout()
plt.show()

# check this code for data set number 4 it links SIR and agent based